<a href="https://colab.research.google.com/github/josipbencic/kaggle/blob/master/predict_future_sales_josip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0 Technical boilerplate to make the notebook work


In [0]:
from google.colab import drive

ROOT = "/content/drive"     # default location for the drive
PROJECT_PATH = "/content/drive/My Drive/kaggle_tmp"

drive.mount(ROOT)           # we mount the google drive at /content/drive

GIT_TOKEN = "13b32b3481c45a9a7bdd7f223644aa47b1156c42"
GIT_USERNAME = "josipbencic"
GIT_REPOSITORY = "kaggle"

GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"

!git clone "{GIT_PATH}" ./temp      # clone github repository to temp folder
!cp -R ./temp/* "{PROJECT_PATH}"    # move all files/folders in temp folder to folder defined in project path
!rm -rf ./temp                      # remove all the files/folders in temp folder
#!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./   # use remote sync to copy from google drive to local runtime google colab
                                                     # but exclude data folder                                          
!rsync -aP "{PROJECT_PATH}"/* ./

!pip install category_encoders
!pip install pycuda
!pip install scikit-cuda

In [0]:
## Inspect collab machine
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

### 1 Thoughts, ideas, conclusions






Ideas

- aggregate items per category instead of per item_id
   - there are only ~80 categories, while 20k items
   - this will reduce the dimensionality in the final version and make
       the model more interpretable, albeit less accurate
   - in that case, what to do with the prices!!??
      - Around 15.5k of items have changed their price over time, ~70%
   - can do 2 versions and then cross validate:
       - with averaged and with summed volumes per item
   - the alternative is to use this column as a latent variable
- battle non-stationarity by building a model only with items that are
   still being sold
- add item price and volume history in the past months as columns
- add a column that asks if the item existed a year ago
- add a column that says how many items of such were sold last year
   in that month
- add last month's sales
- add a few last months' price
- try to backtrade the model to optimize for the parameters

Conclusions
- it's better to model with logarithm of prices rather than with the price itself
  - actually, prices are fucking up everything
  - except the current price, that's useful


### 2 Includes and data loading

In [44]:
%%time
from typing import List, Set, Dict, Tuple, Optional
import argparse
import sys
import os
import csv
import time
import math
from datetime import date
from math import floor

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import category_encoders as ce
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse as mse
#from statsmodels.multivariate.pca import PCA
from sklearn.decomposition import PCA

import pycuda.autoinit
import pycuda.gpuarray as gpuarray
import numpy as np
import skcuda.linalg as linalg
from skcuda.linalg import PCA as cuPCA


file_prefix = './data/'

### Load the datasets from disk
itemCategories=pd.read_csv(file_prefix + 'item_categories.csv')
shops=pd.read_csv(file_prefix + 'shops.csv')
test=pd.read_csv(file_prefix + 'test.csv')
sales_original = pd.read_csv(file_prefix + 'sales_train.csv')

items = pd.read_csv(file_prefix + 'items.csv')
items = items.drop('item_name', axis = 1)
items.set_index('item_id', inplace = True)

CPU times: user 1.14 s, sys: 61.8 ms, total: 1.2 s
Wall time: 1.2 s


### 3 Data Cleaning, Feature Engineering

In [71]:
%%time
SPAN = 5
sales = sales_original.copy()

def build(lo, hi): # [lo, hi)
  data = sales[
      (sales.date_block_num < hi) & (sales.date_block_num >= lo)
    ].groupby(['date_block_num', 'shop_id', 'item_id']).agg(
      sold = pd.NamedAgg(column = 'item_cnt_day', aggfunc = 'sum'),
      price = pd.NamedAgg(column = 'item_price', aggfunc = 'first')
    )
  data.reset_index(drop = False, inplace = True)
  
  ## Append item_id category variable from the other dataset
  data.set_index('item_id', drop = True, inplace = True)
  data = data.join(items, on = 'item_id')
  data.reset_index(inplace = True, drop = False)


  ### append historic data for up to SPAN months before
  aggs = data.copy()
  aggs.drop(inplace = True, columns = ['item_category_id'])
  aggs.rename(columns = {'date_block_num': 'date_block_before'}, inplace = True)
  aggs.set_index(
      ['item_id', 'shop_id', 'date_block_before'], inplace = True, drop = True)

  for it in range(1, SPAN + 1):
    data['date_block_before'] = data.date_block_num - it
    ## Here is where we filter for the first month/s to always have all features
    ## (some features are computed back in time, eg. last month's price)
    data = data[data.date_block_before >= 0]
    data = data.join(
        aggs,
        on = ['item_id', 'shop_id', 'date_block_before'],
        rsuffix = ('_' + str(it))
    )
    data.drop('date_block_before', axis = 1, inplace = True)

    #### Some combinations of shop-item have 0 sales in some months
    ## those items that haven't had any sales in the given slots are assigned 0
    ## instead of NaN
    data['sold_' + str(it)] = data['sold_' + str(it)].fillna(0)
    ## If the price wasn't present at the time:
    ##   - first, fill backwards in time the last price (works with multiple prices)
    ##   - then fill front
    data['price_' + str(it)] = data['price_' + str(it)].fillna(method = 'backfill')
    data['price_' + str(it)] = data['price_' + str(it)].fillna(method = 'ffill')
    data['price_' + str(it) + '_log'] = np.log(data['price_' + str(it)].values)
    data = data.drop('price_' + str(it), axis = 1)
  data['price_log'] = np.log(data['price'].values)

  data = data.drop(['date_block_num'], axis = 1)
  data.reset_index(inplace = True, drop = True)
  return data

CPU times: user 53 ms, sys: 2.09 ms, total: 55.1 ms
Wall time: 54.7 ms


In [88]:
%%time
## Separate and clean the data

df = build(0, 32)
cv = build(32 - SPAN, 33)

### Encoding

## One-hot-encoding for item_id, shop_id, item_category_id
## drop_invariant has to be True
##    it means the constant columns will be dropped
##    this will prevent the normal equations from failing - the matrix won't
##    be singular
enc = ce.BinaryEncoder(cols=['item_id', 'shop_id', 'item_category_id'],
                           drop_invariant = True)
enc.fit(df.loc[:, df.columns != 'sold'], df.sold)
X_enc = enc.transform(df.loc[:, df.columns != 'sold'])
Y_enc = df.sold


X_cv_enc = enc.transform(cv.loc[:, cv.columns != 'sold'])
Y_cv_enc = cv.sold


## PCA
#pca = cuPCA(epsilon = 1e-2)
#X_gpu = gpuarray.GPUArray(X_enc.shape, np.float64, order="F")
#X_gpu.set(X_enc.values)
#Tmp_gpu = pca.fit_transform(X_gpu)
#X = Tmp_gpu.get()

pca = PCA(n_components = 4)
pca.fit(X_enc)


X = pca.transform(X_enc)
X_cv = pca.transform(X_cv_enc)

CPU times: user 18.8 s, sys: 4.03 s, total: 22.8 s
Wall time: 16 s


In [89]:
%%time
## Model building

X = sm.add_constant(X) ## add offset

model = sm.OLS(Y_enc, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                   sold   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.540
Method:                 Least Squares   F-statistic:                 3.671e+05
Date:                Wed, 06 May 2020   Prob (F-statistic):               0.00
Time:                        00:29:57   Log-Likelihood:            -4.0254e+06
No. Observations:             1252845   AIC:                         8.051e+06
Df Residuals:                 1252840   BIC:                         8.051e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3015      0.005    428.367      0.0

### 4 Model Checking - Cross Validation and other


In [0]:
X_cv = sm.add_constant(X_cv)
Y_cv = Y_cv_enc
predictions = model.predict(X_cv)

In [92]:
## PCA - only 4 components, R^2-adj is 540
math.sqrt(mse(predictions, Y_cv)), model.mse_total

(9.624926192000991, 78.54915343910456)

In [87]:
pca.explained_variance_ratio_ # all components

array([9.99859051e-01, 9.85166431e-05, 1.51216558e-05, 9.79375766e-06,
       7.46016112e-06, 5.60831288e-06, 1.15757877e-06, 2.41604750e-07,
       1.83186207e-07, 1.66403712e-07, 1.53900981e-07, 1.52972389e-07,
       1.51264111e-07, 1.47962390e-07, 1.20384548e-07, 1.18147362e-07,
       1.11373490e-07, 1.07894802e-07, 1.04913257e-07, 1.03871595e-07,
       1.03048679e-07, 1.01921543e-07, 9.98800932e-08, 9.77417958e-08,
       9.58727256e-08, 9.49792107e-08, 9.01568247e-08, 8.84309143e-08,
       8.65652209e-08, 8.27019133e-08, 8.06994078e-08, 7.94830824e-08,
       7.56971747e-08, 5.16895265e-08, 4.67880647e-08, 4.14043426e-08,
       3.93009606e-08, 3.47002729e-08, 2.69710588e-08, 9.10177329e-09])

In [85]:
## PCA with all components, R^2- adj 545
math.sqrt(mse(predictions, Y_cv)), model.mse_total

(9.62031692576499, 78.54915343910456)

In [81]:
## PCA with 12 components, R^2- adj 544
math.sqrt(mse(predictions, Y_cv)), model.mse_total

(9.631935077264021, 78.54915343910456)

In [0]:
math.sqrt(mse(predictions, Y_cv)), model.mse_total

(17.165870950393856, 78.54915343910456)

In [0]:
math.sqrt(mse(predictions, Y_cv)), model.mse_total

(17.166284993830292, 78.54915343910456)

In [0]:
math.sqrt(mse(predictions, Y_cv)), model.mse_total

(17.156509971134057, 73.88989502150442)

In [0]:
#plt.matshow(df.loc[:, df.columns.isin(['price_log', 'price_1_log', 'sold_1', 'sold_2', 'sold_3', 'price_2_log', 'sold'])].corr())
#plt.show()

sns.pairplot(df)


### Extra: Random code bits


In [0]:
## don't delete, this unstack idea is top shit and will be needed for sure
def agg_months_as_columns(sales):
  df['month'] = df.time.dt.month
  df = pd.DataFrame(sales.groupby(['month', 'shop_id', 'item_id', \
                                   'item_price'])['item_cnt_day'].sum())
  df = df.unstack(level = 0)
  df = df.fillna(0)
  df.columns = df.columns.get_level_values(1)
  df.reset_index(drop = False, inplace = True)
  df.columns = ['shop_id', 'item_price','Jan', 'Feb',\
                'Mar',  'Apr', 'May',  'Jun', 'Jul',  \
                'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
  return df

### Forward selection with statsmodels
def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model
